In [1]:
#Importing Libraries
import os
from os import listdir
from os.path import isfile, join
from io import StringIO
import pandas as pd
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()
from spacy.matcher import PhraseMatcher

In [2]:
#parsing pdf to text
from tika import parser
raw = parser.from_file("resumes/pv_resume_final.pdf")
text = raw['content']

In [3]:
text

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \nName: Priyanshi Vyas Father’s Name: Mr. P.K Vyas \n\nCollege ID: 170320 Programme: B.Tech. \n\nBranch: Computer Science & Engineering Nationality: Indian \n\nDate of Birth: 01/04/1999 Gender: Female \n\nE-mail ID: anshi99vyas@gmail.com \n \n\nPermanent Address: CF-74, Zinc Colony, \n \n\nHurda, Bhilwara (Rajasthan)  \n\nCAREER OBJECTIVE                                                                 \n \n\n \n\nTo obtain a responsible, challenging, and awarding position in an organization with \n\nan established track record where my skills, abilities and technical abilities can be \n\nutilized. \n\n \n\n AREAS OF INTEREST  \n \n\n• Operating System \n\n• Computer Networks \n \n\n EDUCATIONAL DETAILS  \n\n \n\nQualification Institute Board Year of \nPassing \n\nCGPA / Marks \n\n% \n\n5\nth Semester\n\n Mody University of \nScience \n\nState University 2021 6.92 \n\nB.Tech. & Technology, \nLakshman

In [4]:
#function that does phrase matching and builds a candidate profile
def create_profile(file):
  #below is the csv where we have all the keywords, you can customize your own
        keyword_dict = pd.read_csv('skills_classify.csv',encoding='latin1')
        language = [nlp(text) for text in keyword_dict['Languages'].dropna(axis = 0)]
        big_data = [nlp(text) for text in keyword_dict['BigDataAnalysis'].dropna(axis = 0)]
        coding = [nlp(text) for text in keyword_dict['CodingAndProgramming'].dropna(axis = 0)]
        data_science = [nlp(text) for text in keyword_dict['DataScience'].dropna(axis = 0)]
        devops = [nlp(text) for text in keyword_dict['DevOps'].dropna(axis = 0)]
        cloud = [nlp(text) for text in keyword_dict['CloudComputing'].dropna(axis = 0)]
        ops = [nlp(text) for text in keyword_dict['OperatingSystem'].dropna(axis = 0)]
        web = [nlp(text) for text in keyword_dict['WebDevelopement'].dropna(axis = 0)]
        dbms = [nlp(text) for text in keyword_dict['DBMS'].dropna(axis = 0)]
        app = [nlp(text) for text in keyword_dict['AppDevelopment'].dropna(axis = 0)]
        security = [nlp(text) for text in keyword_dict['Security'].dropna(axis = 0)]
        other = [nlp(text) for text in keyword_dict['Others'].dropna(axis = 0)]

        matcher = PhraseMatcher(nlp.vocab)
        matcher.add('Languages', None, *language)
        matcher.add('BigDataAnalysis', None, *big_data)
        matcher.add('CodingAndProgramming', None, *coding)
        matcher.add('DataScience', None, *data_science)
        matcher.add('DevOps', None, *devops)
        matcher.add('CloudComputing', None, *cloud)
        matcher.add('OperatingSystem', None, *ops)
        matcher.add('WebDevelopement', None, *web)
        matcher.add('DBMS', None, *dbms)
        matcher.add('AppDevelopment', None, *app)
        matcher.add('Security', None, *security)
        matcher.add('Others', None, *other)
        doc = nlp(text)
        #print(doc)
        d = []  
        matches = matcher(doc)
        for match_id, start, end in matches:
            rule_id = nlp.vocab.strings[match_id]  # get the unicode ID, i.e. 'COLOR'
            span = doc[start : end]  # get the matched slice of the doc
            d.append((rule_id, span.text))      
        keywords = "\n".join(f'{i[0]} {i[1]} ({j})' for i,j in Counter(d).items())


      ## convertimg string of keywords to dataframe

        df = pd.read_csv(StringIO(keywords),names = ['Keywords_List'])
        df1 = pd.DataFrame(df.Keywords_List.str.split(' ',1).tolist(),columns = ['Subject','Keyword'])
        df2 = pd.DataFrame(df1.Keyword.str.split('(',1).tolist(),columns = ['Keyword', 'Count'])
        df3 = pd.concat([df1['Subject'],df2['Keyword'], df2['Count']], axis =1) 
        df3['Count'] = df3['Count'].apply(lambda x: x.rstrip(")"))
        #print(df1['Subject'])


        base = os.path.basename(file)
        filename = os.path.splitext(base)[0]

        name = filename.split('_')
        name2 = name[0]
        name2 = name2.lower()
      ## converting str to dataframe
        name3 = pd.read_csv(StringIO(name2),names = ['CandidateName'])

        dataf = pd.concat([name3['CandidateName'], df3['Subject'], df3['Keyword'], df3['Count']], axis = 1)
        dataf['CandidateName'].fillna(dataf['CandidateName'].iloc[0], inplace = True)

        return(dataf)

    #function ends



In [5]:
final_database=pd.DataFrame()
dat = create_profile("/resumes/pv_resume_final.pdf")
final_database = final_database.append(dat)

In [6]:
final_database

,CandidateName,Subject,Keyword,Count
0,pv,OperatingSystem,Linux,3
1,pv,DevOps,Ansible,2
2,pv,DevOps,Docker,3
3,pv,DataScience,Computer Vision,2
4,pv,DataScience,Machine Learning,1
5,pv,CodingAndProgramming,Computing,1
6,pv,OperatingSystem,Windows,1
7,pv,CodingAndProgramming,Programming,1
8,pv,Languages,C++,1
9,pv,WebDevelopement,HTML,1


In [7]:
subcol = final_database["Keyword"].unique()

In [8]:
subcol

array(['Linux ', 'Ansible ', 'Docker ', 'Computer Vision ',
       'Machine Learning ', 'Computing ', 'Windows ', 'Programming ',
       'C++ ', 'HTML '], dtype=object)

In [9]:
keyword_dict = pd.read_csv('skills_classify.csv',encoding='latin1')

In [10]:
final_database2 = final_database['Keyword'].groupby([final_database['CandidateName'], final_database['Subject']]).count().unstack()
final_database2.reset_index(inplace = True)
final_database2.fillna(0,inplace=True)

if(len(final_database2.iloc[0]) < len(keyword_dict.iloc[0])):
    i=0
    for i in keyword_dict:
        if i not in final_database2:
            final_database2[i] = 0


new_data = final_database2.iloc[:,1:]
new_data.index = final_database2['CandidateName']
#execute the below line if you want to see the candidate profile in a csv format
sample2=new_data.to_csv('testingData.csv')


In [11]:
df = pd.read_csv('testingData.csv')

In [12]:
df

,CandidateName,CodingAndProgramming,DataScience,DevOps,Languages,OperatingSystem,WebDevelopement,AppDevelopment,BigDataAnalysis,CloudComputing,DBMS,Others,Security
0,pv,2,2,2,1,2,1,0,0,0,0,0,0


# Extracting CGPA

In [13]:
from tika import parser
raw = parser.from_file("resumes/pv_resume_final.pdf")
text = raw['content']

In [14]:
text

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \nName: Priyanshi Vyas Father’s Name: Mr. P.K Vyas \n\nCollege ID: 170320 Programme: B.Tech. \n\nBranch: Computer Science & Engineering Nationality: Indian \n\nDate of Birth: 01/04/1999 Gender: Female \n\nE-mail ID: anshi99vyas@gmail.com \n \n\nPermanent Address: CF-74, Zinc Colony, \n \n\nHurda, Bhilwara (Rajasthan)  \n\nCAREER OBJECTIVE                                                                 \n \n\n \n\nTo obtain a responsible, challenging, and awarding position in an organization with \n\nan established track record where my skills, abilities and technical abilities can be \n\nutilized. \n\n \n\n AREAS OF INTEREST  \n \n\n• Operating System \n\n• Computer Networks \n \n\n EDUCATIONAL DETAILS  \n\n \n\nQualification Institute Board Year of \nPassing \n\nCGPA / Marks \n\n% \n\n5\nth Semester\n\n Mody University of \nScience \n\nState University 2021 6.92 \n\nB.Tech. & Technology, \nLakshman

In [15]:
text = text.lower()

In [16]:
import re
if (text.find('cgpa') != -1):
    index = text.index('cgpa')
    text = text.strip(' ') 
    #print(text)
    #num = re.findall(r'\d+.\d+', text[index-10:len(text)])
    num = re.findall(r'\d+(?:\.\d*)', text[index-10:len(text)])
else:
    num = 0
    

In [17]:
num[0]

'6.92'

In [18]:
df['CGPA'] = num[0]

In [19]:
df

,CandidateName,CodingAndProgramming,DataScience,DevOps,Languages,OperatingSystem,WebDevelopement,AppDevelopment,BigDataAnalysis,CloudComputing,DBMS,Others,Security,CGPA
0,pv,2,2,2,1,2,1,0,0,0,0,0,0,6.92


In [20]:
X_test =df.drop(['CandidateName'],axis=1)

In [21]:
import pickle

In [22]:
X_test

,CodingAndProgramming,DataScience,DevOps,Languages,OperatingSystem,WebDevelopement,AppDevelopment,BigDataAnalysis,CloudComputing,DBMS,Others,Security,CGPA
0,2,2,2,1,2,1,0,0,0,0,0,0,6.92


In [23]:
loaded_model = pickle.load(open("cv_analysis_model.sav", 'rb'))
result = loaded_model.predict(X_test)
if result == 0:
    df['Placement_status'] = result
    print("Sorry! Work on your skills")
else:
    df['Placement_status'] = result
    print("Great Work! You have Amazing SkillSet")

Great Work! You have Amazing SkillSet


In [24]:
df

,CandidateName,CodingAndProgramming,DataScience,DevOps,Languages,OperatingSystem,WebDevelopement,AppDevelopment,BigDataAnalysis,CloudComputing,DBMS,Others,Security,CGPA,Placement_status
0,pv,2,2,2,1,2,1,0,0,0,0,0,0,6.92,1


In [26]:
df1 = pd.read_csv("final_Dataset.csv")

In [27]:
df1

,CandidateName,Languages,BigDataAnalysis,CodingAndProgramming,DataScience,DevOps,CloudComputing,OperatingSystem,WebDevelopement,DBMS,AppDevelopment,Security,Others,CGPA,Placement_status
0,Abi sharma,2,0,0,0,0,0,0,0,0,1,0,0,6.99,No
1,Anjali Sharma,2,0,1,0,0,0,0,2,0,0,0,0,7.31,No
2,Apoorv Upadhyay,4,0,2,1,0,0,0,4,0,0,0,0,7.80,YES
3,Arti,3,0,1,1,0,0,0,0,0,0,0,0,8.34,No
4,Ayush Paldecha,4,0,1,3,0,0,0,5,1,0,0,1,NaN,No
5,Bhavya Narang,3,0,1,0,3,0,0,0,0,0,0,0,9.23,YES
6,Bhoomika Gautam,2,0,0,0,0,0,0,1,0,3,0,0,8.00,No
7,bhupendrasinghtanwar786@gmail.com,2,1,1,0,0,0,1,0,0,0,0,0,7.33,YES
8,Dikshita Jain,5,1,2,0,0,0,0,10,0,1,0,0,8.82,YES
9,Dinesh Chouhan,3,0,1,0,0,0,0,2,0,0,0,0,7.24,YES


In [28]:
from sklearn.preprocessing import LabelEncoder

# Make copy to avoid changing original data 
object_cols=['Placement_status']

# Apply label encoder to each column with categorical data
label_encoder = LabelEncoder()
for col in object_cols:
    df1[col] = label_encoder.fit_transform(df1[col])
df1

,CandidateName,Languages,BigDataAnalysis,CodingAndProgramming,DataScience,DevOps,CloudComputing,OperatingSystem,WebDevelopement,DBMS,AppDevelopment,Security,Others,CGPA,Placement_status
0,Abi sharma,2,0,0,0,0,0,0,0,0,1,0,0,6.99,0
1,Anjali Sharma,2,0,1,0,0,0,0,2,0,0,0,0,7.31,0
2,Apoorv Upadhyay,4,0,2,1,0,0,0,4,0,0,0,0,7.80,1
3,Arti,3,0,1,1,0,0,0,0,0,0,0,0,8.34,0
4,Ayush Paldecha,4,0,1,3,0,0,0,5,1,0,0,1,NaN,0
5,Bhavya Narang,3,0,1,0,3,0,0,0,0,0,0,0,9.23,1
6,Bhoomika Gautam,2,0,0,0,0,0,0,1,0,3,0,0,8.00,0
7,bhupendrasinghtanwar786@gmail.com,2,1,1,0,0,0,1,0,0,0,0,0,7.33,1
8,Dikshita Jain,5,1,2,0,0,0,0,10,0,1,0,0,8.82,1
9,Dinesh Chouhan,3,0,1,0,0,0,0,2,0,0,0,0,7.24,1


In [29]:
df1 = df1.append(df)

In [30]:
df1

,CandidateName,Languages,BigDataAnalysis,CodingAndProgramming,DataScience,DevOps,CloudComputing,OperatingSystem,WebDevelopement,DBMS,AppDevelopment,Security,Others,CGPA,Placement_status
0,Abi sharma,2,0,0,0,0,0,0,0,0,1,0,0,6.99,0
1,Anjali Sharma,2,0,1,0,0,0,0,2,0,0,0,0,7.31,0
2,Apoorv Upadhyay,4,0,2,1,0,0,0,4,0,0,0,0,7.8,1
3,Arti,3,0,1,1,0,0,0,0,0,0,0,0,8.34,0
4,Ayush Paldecha,4,0,1,3,0,0,0,5,1,0,0,1,NaN,0
5,Bhavya Narang,3,0,1,0,3,0,0,0,0,0,0,0,9.23,1
6,Bhoomika Gautam,2,0,0,0,0,0,0,1,0,3,0,0,8.0,0
7,bhupendrasinghtanwar786@gmail.com,2,1,1,0,0,0,1,0,0,0,0,0,7.33,1
8,Dikshita Jain,5,1,2,0,0,0,0,10,0,1,0,0,8.82,1
9,Dinesh Chouhan,3,0,1,0,0,0,0,2,0,0,0,0,7.24,1


In [31]:
df1.columns

Index(['CandidateName', 'Languages', 'BigDataAnalysis', 'CodingAndProgramming',
       'DataScience', 'DevOps', 'CloudComputing', 'OperatingSystem',
       'WebDevelopement', 'DBMS', 'AppDevelopment', 'Security', 'Others',
       'CGPA', 'Placement_status'],
      dtype='object')

In [32]:
from sklearn.impute import SimpleImputer
def imputation(df1,variable,median):
    df1[variable] = df1[variable].fillna(median)

In [33]:
median = df1.CGPA.median()

In [34]:
imputation(df1,'CGPA',median)

In [35]:
df1

,CandidateName,Languages,BigDataAnalysis,CodingAndProgramming,DataScience,DevOps,CloudComputing,OperatingSystem,WebDevelopement,DBMS,AppDevelopment,Security,Others,CGPA,Placement_status
0,Abi sharma,2,0,0,0,0,0,0,0,0,1,0,0,6.99,0
1,Anjali Sharma,2,0,1,0,0,0,0,2,0,0,0,0,7.31,0
2,Apoorv Upadhyay,4,0,2,1,0,0,0,4,0,0,0,0,7.8,1
3,Arti,3,0,1,1,0,0,0,0,0,0,0,0,8.34,0
4,Ayush Paldecha,4,0,1,3,0,0,0,5,1,0,0,1,7.705,0
5,Bhavya Narang,3,0,1,0,3,0,0,0,0,0,0,0,9.23,1
6,Bhoomika Gautam,2,0,0,0,0,0,0,1,0,3,0,0,8.0,0
7,bhupendrasinghtanwar786@gmail.com,2,1,1,0,0,0,1,0,0,0,0,0,7.33,1
8,Dikshita Jain,5,1,2,0,0,0,0,10,0,1,0,0,8.82,1
9,Dinesh Chouhan,3,0,1,0,0,0,0,2,0,0,0,0,7.24,1


In [36]:
X=df1.drop(['Placement_status','CandidateName'],axis=1)
y=df1.Placement_status

In [37]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,train_size=0.8,random_state=1)

In [38]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [39]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)*100

50.0

In [40]:
filename = "cv_analysis_model.sav"
pickle.dump(model, open(filename, 'wb'))

In [41]:
import pandas as pd

In [42]:
df2 = pd.read_csv("testingData.csv")

In [43]:
df1.to_csv("finalDatasets.csv",index=False)

In [44]:
df

,CandidateName,CodingAndProgramming,DataScience,DevOps,Languages,OperatingSystem,WebDevelopement,AppDevelopment,BigDataAnalysis,CloudComputing,DBMS,Others,Security,CGPA,Placement_status
0,pv,2,2,2,1,2,1,0,0,0,0,0,0,6.92,1


In [45]:
df.columns

Index(['CandidateName', 'CodingAndProgramming', 'DataScience', 'DevOps',
       'Languages', 'OperatingSystem', 'WebDevelopement', 'AppDevelopment',
       'BigDataAnalysis', 'CloudComputing', 'DBMS', 'Others', 'Security',
       'CGPA', 'Placement_status'],
      dtype='object')

In [46]:
df = df.drop(['CodingAndProgramming','DataScience','DevOps','Languages','OperatingSystem', 'WebDevelopement','AppDevelopment','BigDataAnalysis', 'CloudComputing', 'DBMS', 'Others', 'Security'],axis=1)

In [47]:
len(subcol)

10

In [48]:
subcol = subcol.tolist()

In [49]:
subcoldf = pd.DataFrame({"SkillSet" : [subcol]})

In [50]:
df["SkillSet"] = subcoldf["SkillSet"]

In [53]:
df

,CandidateName,CGPA,Placement_status,SkillSet
0,pv,6.92,1,"[Linux , Ansible , Docker , Computer Vision , ..."


In [52]:
df.to_csv('DisplayData.csv', mode='a', header=False , index = False)

In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('finalDatasets.csv')

In [5]:
data.describe()

,Languages,BigDataAnalysis,CodingAndProgramming,DataScience,DevOps,CloudComputing,OperatingSystem,WebDevelopement,DBMS,AppDevelopment,Security,Others,CGPA,Placement_status
count,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.00000,75.000000
mean,2.493333,0.413333,1.480000,0.880000,1.013333,0.333333,0.720000,2.106667,0.386667,0.466667,0.040000,0.613333,8.37200,0.720000
std,1.388888,0.772734,1.166654,1.470062,1.614908,0.684448,0.994036,2.549898,0.714584,0.776919,0.197279,1.137723,9.87178,0.452022
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
25%,2.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.95500,0.000000
50%,2.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,7.70500,1.000000
75%,3.000000,1.000000,2.000000,1.000000,2.000000,0.000000,2.000000,3.000000,1.000000,1.000000,0.000000,1.000000,8.00000,1.000000
max,7.000000,3.000000,4.000000,7.000000,5.000000,3.000000,3.000000,13.000000,3.000000,4.000000,1.000000,8.000000,71.00000,1.000000
